# Variational Autoencoder for Regression 

- Paper https://arxiv.org/abs/1904.05948

- Repository https://github.com/QingyuZhao/VAE-for-Regression


In [13]:
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F


### Load Example Data

In [14]:
# Load Toy Example Data
training_feature = np.loadtxt('data/X.txt')
training_feature.shape

Y = np.loadtxt('data/Y.txt')
ground_truth_r = Y

np.random.seed(seed=0)

original_dim = training_feature.shape[1]
num_train = training_feature.shape[0]

In [15]:
def sampling(args):
    '''
    Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    Returns:
        z (tensor): sampled latent vector
    '''
    mean, log_var = args
    epsilon = torch.randn_like(mean) 
    return mean + torch.exp(0.5*log_var)*epsilon 

### Build VAE Regression Model 

In [16]:
class Encoder(nn.Module):
    def __init__(self, input_shape_x, intermediate_dim, latent_dim):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p=0.25)
        self.fc1 = nn.Linear(input_shape_x, 128)
        self.act1 = nn.Tanh()
        self.fc2 = nn.Linear(128, intermediate_dim)
        self.act2 = nn.Tanh() 
        
        # posterior on Y; probabilistic regressor 
        self.r_mean = nn.Linear(intermediate_dim, 1)
        self.r_logvar = nn.Linear(intermediate_dim, 1) 

        # q(z|x) 
        self.z_mean = nn.Linear(intermediate_dim, latent_dim)
        self.z_logvar = nn.Linear(intermediate_dim, latent_dim)

        # latent generator 
        self.gen_z = nn.Linear(1, latent_dim)


    def forward(self, x):
        x = self.dropout(x)
        x = self.act1(self.fc1(x))
        x = self.act2(self.fc2(x))

        r_mean = self.r_mean(x)
        r_logvar = self.r_logvar(x)

        z_mean = self.z_mean(x)
        z_logvar = self.z_logvar(x)

        # reparameterization trick
        r = sampling(self.r_mean, self.r_logvar)
        z = sampling(self.z_mean, self.z_logvar)

        pz_mean = self.gen_z(r) 

        return r_mean, r_logvar, r, z_mean, z_logvar, z, pz_mean


In [17]:
class Decoder(nn.Module):
    def __init__(self, input_shape_x, intermediate_dim, latent_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, intermediate_dim)
        self.act1 = nn.Tanh()
        self.fc2 = nn.Linear(intermediate_dim, 128)
        self.act2 = nn.Tanh()
        self.fc3 = nn.Linear(128, input_shape_x)

    
    def forward(self, x):
        x = self.act1(self.fc1(x))
        x = self.act2(self.fc2(x))
        x = self.fc3(x)
        return x 

In [ ]:
encoder = Encoder(original_dim, 32, 8)
decoder = Decoder(original_dim, 32, 8) 